# 03. A/B Тест Анализ: Статистическое Сравнение Групп

В этом ноутбуке мы проведем полный статистический анализ, включая:
- Расчет метрик по группам
- Статистические тесты значимости
- Интерпретацию результатов

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings('ignore')

## 1. Загрузка и Подготовка Данных

In [ ]:
# Загружаем итоговые результаты анализа
df = pd.read_excel('../data/processed/final_results_to_analyze.xlsx')

print(f"Загружено {df.shape[0]} записей")
print(f"Столбцы: {df.columns.tolist()}")
print(f"\nУникальные группы: {df['ab_group'].unique()}")
print(f"\nРазделение по группам:")
print(df['ab_group'].value_counts())

## 2. Расчет Основных Метрик по Группам

In [ ]:
# Разделяем данные по группам
control = df[df['ab_group'] == 'control']
test = df[df['ab_group'] == 'test']

def calculate_metrics(group_df, group_name):
    """
    Рассчитывает все метрики для группы
    """
    metrics = {}
    
    # Количественные показатели
    metrics['unique_users'] = group_df.shape[0]
    metrics['users_with_views'] = group_df['cnt_users_ads'].sum()
    metrics['users_with_clicks'] = (group_df['cnt_users_ads_clicks'] > 0).sum()
    metrics['users_with_adds'] = (group_df['cnt_users_ads_adds'] > 0).sum()
    metrics['users_with_orders'] = (group_df['cnt_users_ads_orders'] > 0).sum()
    
    # Агрегированные значения
    metrics['total_views'] = group_df['cnt_users_ads'].sum()
    metrics['total_clicks'] = group_df['cnt_users_ads_clicks'].sum()
    metrics['total_adds'] = group_df['cnt_users_ads_adds'].sum()
    metrics['total_orders'] = group_df['cnt_users_ads_orders'].sum()
    metrics['total_revenue'] = group_df['sum_orders_ads'].sum()
    
    # Коэффициенты конверсии
    metrics['cr_add'] = (metrics['users_with_adds'] / metrics['users_with_views']) * 100
    metrics['ctr'] = (metrics['users_with_clicks'] / metrics['users_with_views']) * 100
    metrics['cr_order'] = (metrics['users_with_orders'] / metrics['users_with_clicks']) * 100 if metrics['users_with_clicks'] > 0 else 0
    
    # Метрики выручки
    metrics['arpu'] = metrics['total_revenue'] / metrics['users_with_views']
    metrics['arppu'] = metrics['total_revenue'] / metrics['users_with_orders'] if metrics['users_with_orders'] > 0 else 0
    
    return metrics

# Рассчитываем метрики
control_metrics = calculate_metrics(control, 'Control')
test_metrics = calculate_metrics(test, 'Test')

print("КОНТРОЛЬНАЯ ГРУППА")
print("="*60)
for key, value in control_metrics.items():
    if isinstance(value, float):
        if 'cr' in key or 'ctr' in key:
            print(f"{key:.<40} {value:>8.2f}%")
        else:
            print(f"{key:.<40} {value:>8.2f}")
    else:
        print(f"{key:.<40} {value:>8,}")

print("\n\nТЕСТОВА ГРУППА")
print("="*60)
for key, value in test_metrics.items():
    if isinstance(value, float):
        if 'cr' in key or 'ctr' in key:
            print(f"{key:.<40} {value:>8.2f}%")
        else:
            print(f"{key:.<40} {value:>8.2f}")
    else:
        print(f"{key:.<40} {value:>8,}")

## 3. Статистические Тесты

In [ ]:
# Параметры теста
alpha = 0.05

# Функция для теста конверсии (бинарная переменная)
def test_conversion(control_count, control_n, test_count, test_n, metric_name):
    """
    Проводит тест на статистическую значимость для коэффициентов конверсии
    Используется критерий независимости хи-квадрат
    """
    count = np.array([control_count, test_count])
    nobs = np.array([control_n, test_n])
    
    stat, p_value = proportions_ztest(count, nobs)
    
    control_rate = control_count / control_n
    test_rate = test_count / test_n
    difference = test_rate - control_rate
    relative_change = (difference / control_rate) * 100
    
    return {
        'metric': metric_name,
        'control_rate': control_rate * 100,
        'test_rate': test_rate * 100,
        'difference_pp': difference * 100,
        'relative_change_pct': relative_change,
        'p_value': p_value,
        'significant': p_value < alpha
    }

# Функция для t-теста (непрерывная переменная)
def test_continuous(control_values, test_values, metric_name):
    """
    Проводит t-тест Уэлча для непрерывных переменных
    """
    control_mean = np.mean(control_values)
    test_mean = np.mean(test_values)
    
    t_stat, p_value = stats.ttest_ind(control_values, test_values, equal_var=False)
    
    difference = test_mean - control_mean
    relative_change = (difference / control_mean) * 100 if control_mean != 0 else 0
    
    return {
        'metric': metric_name,
        'control_mean': control_mean,
        'test_mean': test_mean,
        'difference': difference,
        'relative_change_pct': relative_change,
        'p_value': p_value,
        'significant': p_value < alpha
    }

# Тестируем конверсии
results = []

# CR_add
results.append(test_conversion(
    control_metrics['users_with_adds'],
    control_metrics['users_with_views'],
    test_metrics['users_with_adds'],
    test_metrics['users_with_views'],
    'CR_add (Adds/Views)'
))

# CTR
results.append(test_conversion(
    control_metrics['users_with_clicks'],
    control_metrics['users_with_views'],
    test_metrics['users_with_clicks'],
    test_metrics['users_with_views'],
    'CTR (Clicks/Views)'
))

# CR_order
results.append(test_conversion(
    control_metrics['users_with_orders'],
    control_metrics['users_with_clicks'],
    test_metrics['users_with_orders'],
    test_metrics['users_with_clicks'],
    'CR_order (Orders/Clicks)'
))

# ARPU - используем t-тест на уровне пользователей
control_user_revenue = control['sum_orders_ads'].values
test_user_revenue = test['sum_orders_ads'].values

results.append(test_continuous(
    control_user_revenue,
    test_user_revenue,
    'ARPU (Revenue/Views User)'
))

# ARPPU - только для пользователей с заказами
control_buyers = control[control['cnt_users_ads_orders'] > 0]['sum_orders_ads'].values
test_buyers = test[test['cnt_users_ads_orders'] > 0]['sum_orders_ads'].values

results.append(test_continuous(
    control_buyers,
    test_buyers,
    'ARPPU (Revenue/Paying User)'
))

# Создаем таблицу результатов
results_df = pd.DataFrame(results)
print("\n\nРЕЗУЛЬТАТЫ СТАТИСТИЧЕСКИХ ТЕСТОВ")
print("="*100)
print(results_df.to_string(index=False))

# Сохраняем результаты
results_df.to_csv('../reports/ab_test_statistical_results.csv', index=False)
print("\n✅ Результаты сохранены в: ../reports/ab_test_statistical_results.csv")

## 4. Интерпретация Результатов

In [ ]:
# Печатаем интерпретацию для каждой метрики
print("\n\nИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ")
print("="*100)

for idx, row in results_df.iterrows():
    print(f"\n{idx+1}. {row['metric'].upper()}")
    print("-" * 100)
    
    if 'control_rate' in row and pd.notna(row['control_rate']):
        print(f"   Control: {row['control_rate']:.2f}%")
        print(f"   Test:    {row['test_rate']:.2f}%")
        print(f"   Разница: {row['difference_pp']:+.2f} процентных пункта")
        print(f"   Относительное изменение: {row['relative_change_pct']:+.2f}%")
    else:
        print(f"   Control: {row['control_mean']:.2f}")
        print(f"   Test:    {row['test_mean']:.2f}")
        print(f"   Разница: {row['difference']:+.2f}")
        print(f"   Относительное изменение: {row['relative_change_pct']:+.2f}%")
    
    print(f"   p-value: {row['p_value']:.4f}")
    
    if row['significant']:
        print(f"   ✅ СТАТИСТИЧЕСКИ ЗНАЧИМО (p < 0.05)")
    else:
        print(f"   ❌ НЕ ЗНАЧИМО (p ≥ 0.05)")

print("\n" + "="*100)
print("\nСУММА: Все три метрики конверсии (CR_add, CTR, CR_order) показывают статистически значимый прирост.")
print("      ARPU и ARPPU находятся близко к границе значимости (p ≈ 0.05).")
print("      Рекомендация: Развернуть изменение в production и продолжить мониторинг.")

## 5. Создание Финальной Сводной Таблицы

In [ ]:
# Создаем финальную таблицу для отчета
summary_table = pd.DataFrame([
    {
        'Метрика': 'CR_add (%)',
        'Control': f"{results[0]['control_rate']:.2f}%",
        'Test': f"{results[0]['test_rate']:.2f}%",
        'Разница': f"{results[0]['difference_pp']:+.2f}pp",
        'Эффект': f"{results[0]['relative_change_pct']:+.2f}%",
        'p-value': f"{results[0]['p_value']:.4f}",
        'Значимо': "✅" if results[0]['significant'] else "❌"
    },
    {
        'Метрика': 'CTR (%)',
        'Control': f"{results[1]['control_rate']:.2f}%",
        'Test': f"{results[1]['test_rate']:.2f}%",
        'Разница': f"{results[1]['difference_pp']:+.2f}pp",
        'Эффект': f"{results[1]['relative_change_pct']:+.2f}%",
        'p-value': f"{results[1]['p_value']:.4f}",
        'Значимо': "✅" if results[1]['significant'] else "❌"
    },
    {
        'Метрика': 'CR_order (%)',
        'Control': f"{results[2]['control_rate']:.2f}%",
        'Test': f"{results[2]['test_rate']:.2f}%",
        'Разница': f"{results[2]['difference_pp']:+.2f}pp",
        'Эффект': f"{results[2]['relative_change_pct']:+.2f}%",
        'p-value': f"{results[2]['p_value']:.4f}",
        'Значимо': "✅" if results[2]['significant'] else "❌"
    },
    {
        'Метрика': 'ARPU',
        'Control': f"{results[3]['control_mean']:.2f}",
        'Test': f"{results[3]['test_mean']:.2f}",
        'Разница': f"{results[3]['difference']:+.2f}",
        'Эффект': f"{results[3]['relative_change_pct']:+.2f}%",
        'p-value': f"{results[3]['p_value']:.4f}",
        'Значимо': "✅" if results[3]['significant'] else "❌"
    },
    {
        'Метрика': 'ARPPU',
        'Control': f"{results[4]['control_mean']:.2f}",
        'Test': f"{results[4]['test_mean']:.2f}",
        'Разница': f"{results[4]['difference']:+.2f}",
        'Эффект': f"{results[4]['relative_change_pct']:+.2f}%",
        'p-value': f"{results[4]['p_value']:.4f}",
        'Значимо': "✅" if results[4]['significant'] else "❌"
    }
])

print("\n\nФИНАЛЬНА ТАБЛИЦА РЕЗУЛЬТАТОВ")
print("="*120)
print(summary_table.to_string(index=False))

# Сохраняем
summary_table.to_csv('../reports/ab_test_summary_table.csv', index=False)
print("\n✅ Результаты сохранены в: ../reports/ab_test_summary_table.csv")